In [1]:
## Installing Spotify's API library in Python

# pip install spotipy

In [2]:
## Importing libraries

import sys
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
from datetime import datetime

sys.path.append("/home/tabas/personal-dev/pyprojects")
import pipelines.utils.personal_env as penv

In [3]:
# Importing Spotify Credentials

CLIENT_ID = penv.spotify_client_id
CLIENT_SECRET = penv.spotify_client_secret

In [4]:
# Stablishing Spotify Authentication

auth_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(auth_manager=auth_manager)

In [5]:
# Creating a list of all available markets on Spotify

markets = [
            "AD", "AE", "AG", "AL", #"AM", "AO", "AR", "AT", "AU", "AZ", "BA", "BB", "BD", 
            #"BE", "BF", "BG", "BH", "BI", "BJ", "BN", "BO", "BR", "BS", "BT", "BW", "BY", 
            #"BZ", "CA", "CD", "CG", "CH", "CI", "CL", "CM", "CO", "CR", "CV", "CW", "CY", 
            #"CZ", "DE", "DJ", "DK", "DM", "DO", "DZ", "EC", "EE", "EG", "ES", "ET", "FI", 
            #"FJ", "FM", "FR", "GA", "GB", "GD", "GE", "GH", "GM", "GN", "GQ", "GR", "GT", 
            #"GW", "GY", "HK", "HN", "HR", "HT", "HU", "ID", "IE", "IL", "IN", "IQ", "IS", 
            #"IT", "JM", "JO", "JP", "KE", "KG", "KH", "KI", "KM", "KN", "KR", "KW", "KZ", 
            #"LA", "LB", "LC", "LI", "LK", "LR", "LS", "LT", "LU", "LV", "LY", "MA", "MC", 
            #"MD", "ME", "MG", "MH", "MK", "ML", "MN", "MO", "MR", "MT", "MU", "MV", "MW", 
            #"MX", "MY", "MZ", "NA", "NE", "NG", "NI", "NL", "NO", "NP", "NR", "NZ", "OM", 
            #"PA", "PE", "PG", "PH", "PK", "PL", "PR", "PS", "PT", "PW", "PY", "QA", "RO", 
            #"RS", "RW", "SA", "SB", "SC", "SE", "SG", "SI", "SK", "SL", "SM", "SN", "SR", 
            #"ST", "SV", "SZ", "TD", "TG", "TH", "TJ", "TL", "TN", "TO", "TR", "TT", "TV", 
            #"TW", "TZ", "UA", "UG", "US", "UY", "UZ", "VC", "VE", "VN", "VU", "WS", "XK", 
            #"ZA", "ZM", "ZW"
           ]

### Getting latest releases

In [6]:
## Creating empty DataFrame to append API values after request

releases = pd.DataFrame()

In [7]:
## Creating loop to make GET Request
## The first request gets the list of new Albums released two weeks ago from each market defined above
## Then, it collects the ids of the Artists of each release and makes the second request, 
## which returns the Artist's data

for i in range(len(markets)):
    
    ## The Spotify only returns 50 values per request 
    # (the variables 'limit' and 'batchSize' helps Spotify not crash if the data exceeds )
    
    limit = 50
    offset = 0
    
    while offset < 1000:    # Spotify limit for Search Request is 1000
        
        ## Making GET request of the type search with the tag:'new', that returns the latest Albums

        newReleases = sp.search(q="tag:new", market=markets[i], type="album", limit=limit, offset=offset)
        newReleasesData = pd.DataFrame.from_dict(newReleases['albums']['items'])
        
        releases = pd.concat([releases, newReleasesData])
        releases['extractionTimestamp'] = datetime.today().strftime('%Y-%m-%d %X')
        
        # Incremental addition to offset to return the following pages of data
        
        offset=offset+limit
        
    print("Successfully got request from ", markets[i], "market")

Successfully got request from  AD market
Successfully got request from  AE market
Successfully got request from  AG market
Successfully got request from  AL market


In [8]:
# Returning table info

releases.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4000 entries, 0 to 49
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   album_type              4000 non-null   object
 1   total_tracks            4000 non-null   int64 
 2   is_playable             4000 non-null   bool  
 3   external_urls           4000 non-null   object
 4   href                    4000 non-null   object
 5   id                      4000 non-null   object
 6   images                  4000 non-null   object
 7   name                    4000 non-null   object
 8   release_date            4000 non-null   object
 9   release_date_precision  4000 non-null   object
 10  type                    4000 non-null   object
 11  uri                     4000 non-null   object
 12  artists                 4000 non-null   object
 13  extractionTimestamp     4000 non-null   object
dtypes: bool(1), int64(1), object(12)
memory usage: 441.4+ KB


In [9]:
## Treating some fields (renaming, extracting values, etc)

releases['spotify_url'] = releases['external_urls'].apply(lambda x: x['spotify'] if isinstance(x, dict) else None)

# Here, we are maintaining the id, uri and href in lists to facilitate in case we need to use them on the requests below

releases['artist_uri'] = releases['artists'].apply(lambda artists: [artist['uri'] for artist in artists])
releases['artist_href'] = releases['artists'].apply(lambda artists: [artist['href'] for artist in artists])
releases['artist_id'] = releases['artists'].apply(lambda artists: [artist['id'] for artist in artists])


In [10]:
# Dropping unnecessary and/or treated columns

releases = releases.drop(columns=['external_urls', 'artists'])

# Ordering columns

releases = releases[[
                    'id', 
                    'href', 
                    'uri', 
                    'spotify_url', 
                    'album_type', 
                    'total_tracks', 
                    'is_playable', 
                    'name', 
                    'release_date', 
                    'release_date_precision', 
                    'type', 
                    'artist_id', 
                    'artist_href', 
                    'artist_uri', 
                    'images', 
                    'extractionTimestamp']
            ]

In [19]:
# Removing duplicated release id 

releases = releases[releases.duplicated(subset='id') == False]

In [20]:
releases.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1428 entries, 0 to 37
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   id                      1428 non-null   object
 1   href                    1428 non-null   object
 2   uri                     1428 non-null   object
 3   spotify_url             1428 non-null   object
 4   album_type              1428 non-null   object
 5   total_tracks            1428 non-null   int64 
 6   is_playable             1428 non-null   bool  
 7   name                    1428 non-null   object
 8   release_date            1428 non-null   object
 9   release_date_precision  1428 non-null   object
 10  type                    1428 non-null   object
 11  artist_id               1428 non-null   object
 12  artist_href             1428 non-null   object
 13  artist_uri              1428 non-null   object
 14  images                  1428 non-null   object
 15  extractionT

### Getting artists 

In [113]:
## Creating empty DataFrame to append API values after request

artists = pd.DataFrame()

In [114]:
## Making request to GET Artists' Data

# Here we're accessing the Artist ID to make the loop request below 
# We transforme the arrays into sets to remove duplicates, and then convert it back to lists, so it can be used
# on the API request
        
artistsList =  list(set(releases['artist_id'].explode()))

batchSize = 50 # Spotify limit for Artist Request is 50
        
for j in range(0, len(artistsList), batchSize):
    artistsBatch = artistsList[j:j + batchSize]
    artistsData = sp.artists(artistsBatch)
    artistsData = pd.DataFrame.from_dict(artistsData['artists'])  
    artists = pd.concat([artists, artistsData])
    artists['extractionTimestamp'] = datetime.today().strftime('%Y-%m-%d %X')
    
    print("Successfully got ", round(((j + batchSize)/len(artistsList)) * 100, 2), "% of Artists")

Successfully got  3.32 % of Artists
Successfully got  6.64 % of Artists
Successfully got  9.95 % of Artists
Successfully got  13.27 % of Artists
Successfully got  16.59 % of Artists
Successfully got  19.91 % of Artists
Successfully got  23.22 % of Artists
Successfully got  26.54 % of Artists
Successfully got  29.86 % of Artists
Successfully got  33.18 % of Artists
Successfully got  36.5 % of Artists
Successfully got  39.81 % of Artists
Successfully got  43.13 % of Artists
Successfully got  46.45 % of Artists
Successfully got  49.77 % of Artists
Successfully got  53.09 % of Artists
Successfully got  56.4 % of Artists
Successfully got  59.72 % of Artists
Successfully got  63.04 % of Artists
Successfully got  66.36 % of Artists
Successfully got  69.67 % of Artists
Successfully got  72.99 % of Artists
Successfully got  76.31 % of Artists
Successfully got  79.63 % of Artists
Successfully got  82.95 % of Artists
Successfully got  86.26 % of Artists
Successfully got  89.58 % of Artists
Succes

In [115]:
artists.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1507 entries, 0 to 6
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   external_urls        1507 non-null   object
 1   followers            1507 non-null   object
 2   genres               1507 non-null   object
 3   href                 1507 non-null   object
 4   id                   1507 non-null   object
 5   images               1507 non-null   object
 6   name                 1507 non-null   object
 7   popularity           1507 non-null   int64 
 8   type                 1507 non-null   object
 9   uri                  1507 non-null   object
 10  extractionTimestamp  1507 non-null   object
dtypes: int64(1), object(10)
memory usage: 141.3+ KB


In [122]:
## Returning Artists DataFrame

artists.head()

,id,href,uri,spotify_url,type,name,followers,popularity,genres,images,extractionTimestamp
0,4nCeCLzpxBVKiK4PXR3Xi1,https://api.spotify.com/v1/artists/4nCeCLzpxBV...,spotify:artist:4nCeCLzpxBVKiK4PXR3Xi1,https://open.spotify.com/artist/4nCeCLzpxBVKiK...,artist,Dj Ryan Mpc,556,25,[],[{'url': 'https://i.scdn.co/image/ab6761610000...,2025-02-06 10:58:53
1,6uZv9f2du7TdOlOr5IjLE1,https://api.spotify.com/v1/artists/6uZv9f2du7T...,spotify:artist:6uZv9f2du7TdOlOr5IjLE1,https://open.spotify.com/artist/6uZv9f2du7TdOl...,artist,Shabab Sabri,38479,48,[bollywood],[{'url': 'https://i.scdn.co/image/ab6761610000...,2025-02-06 10:58:53
2,3Pny1kTPzgQoaB5pny2bta,https://api.spotify.com/v1/artists/3Pny1kTPzgQ...,spotify:artist:3Pny1kTPzgQoaB5pny2bta,https://open.spotify.com/artist/3Pny1kTPzgQoaB...,artist,Vishnu Sharma,238,3,"[bhajan, devotional]",[{'url': 'https://i.scdn.co/image/ab67616d0000...,2025-02-06 10:58:53
3,3LGIFvM03aaCTDFYelZT6a,https://api.spotify.com/v1/artists/3LGIFvM03aa...,spotify:artist:3LGIFvM03aaCTDFYelZT6a,https://open.spotify.com/artist/3LGIFvM03aaCTD...,artist,DJ PSICO DE CAXIAS,2780,40,"[funk carioca, brazilian funk]",[{'url': 'https://i.scdn.co/image/ab6761610000...,2025-02-06 10:58:53
4,1pRruDTNcEFpBPvpFZU76o,https://api.spotify.com/v1/artists/1pRruDTNcEF...,spotify:artist:1pRruDTNcEFpBPvpFZU76o,https://open.spotify.com/artist/1pRruDTNcEFpBP...,artist,Rupam Islam,323750,50,[bangla pop],[{'url': 'https://i.scdn.co/image/ab6761610000...,2025-02-06 10:58:53


In [117]:
## Treating some fields (renaming, exploding the dicts, etc)

artists['spotify_url'] = artists['external_urls'].apply(lambda x: x['spotify'] if isinstance(x, dict) else None)
artists['followers'] = artists['followers'].apply(lambda x: x['total'] if isinstance(x, dict) else None)


In [119]:
# Dropping unnecessary and/or treated columns

artists = artists.drop(columns=['external_urls'])

# Ordering columns

artists = artists[[
                    'id', 
                    'href', 
                    'uri', 
                    'spotify_url', 
                    'type',
                    'name', 
                    'followers', 
                    'popularity', 
                    'genres', 
                    'images', 
                    'extractionTimestamp']
]

In [121]:
# Removing duplicated release id 

artists = artists[artists.duplicated(subset='id') == False]

In [126]:
artists.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1507 entries, 0 to 6
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   id                   1507 non-null   object
 1   href                 1507 non-null   object
 2   uri                  1507 non-null   object
 3   spotify_url          1507 non-null   object
 4   type                 1507 non-null   object
 5   name                 1507 non-null   object
 6   followers            1507 non-null   int64 
 7   popularity           1507 non-null   int64 
 8   genres               1507 non-null   object
 9   images               1507 non-null   object
 10  extractionTimestamp  1507 non-null   object
dtypes: int64(2), object(9)
memory usage: 141.3+ KB


In [344]:
## Let's get more data on this script
## Now that we have New Releases information, we can make a request to collect further information about albums and tracks
## Note that a new release can be an album or a single.
## So, to get more data, we're going to make a Request to return more informations about albums (like tracks, for example)
## And then, we are going to make a final request to return information about all tracks (single releases and tracks from the albums)

In [345]:
## Creating empty DataFrame to append API values after request

albums = pd.DataFrame()

In [ ]:
# Again, using set and list to make the Album request below

albumsList = list(set(releases['id']))

# Redefining batchSize variable

batchSize = 20 # Spotify limit for Album Request is 20

for k in range(0, len(albumsList), batchSize):
    
    albumsData = sp.albums(albumsList[k:k + batchSize])
    albumsData = pd.DataFrame.from_dict(albumsData['albums'])
    albums = pd.concat([albums, albumsData])
    albums['extractionTimestamp'] = datetime.today().strftime('%Y-%m-%d %X')
    
    print("Successfully got ", round((k + batchSize) / len(albumsList) * 100, 2), "% of albums")

In [ ]:
albums.info()

In [ ]:
albums.head()

In [349]:
## Treating some fields (renaming, extracting values, etc)

albums['spotify_url'] = pd.json_normalize(albums['external_urls'])

albums['artist_id'] = albums['artists'].apply(lambda artists: [artist['id'] for artist in artists])
albums['artist_href'] = albums['artists'].apply(lambda artists: [artist['href'] for artist in artists])
albums['artist_uri'] = albums['artists'].apply(lambda artists: [artist['uri'] for artist in artists])

albums['track_id'] = pd.json_normalize(albums['tracks'])['items'].apply(lambda x: [track['id'] for track in x] if isinstance(x, list) else [])
albums['track_href'] = pd.json_normalize(albums['tracks'])['items'].apply(lambda x: [track['href'] for track in x] if isinstance(x, list) else [])
albums['track_uri'] = pd.json_normalize(albums['tracks'])['items'].apply(lambda x: [track['uri'] for track in x] if isinstance(x, list) else [])

In [350]:
# Removing unnecessary and/or treated columns

#albums = albums.drop(columns=['external_urls', 'artists', 'tracks', 'genres']) 

# Ordering columns

albums = albums[[
                'id', 
                'href',
                'uri',
                'spotify_url',
                'album_type',
                'total_tracks',  
                'name', 
                'available_markets',
                'release_date', 
                'release_date_precision', 
                'type', 
                'artist_id', 
                'artist_href', 
                'artist_uri', 
                'track_id',
                'track_href',
                'track_uri',
                'popularity',
                'label',
                'copyrights',
                'external_ids',                
                'images', 
                'extractionTimestamp']
            ]


In [351]:
# Removing duplicated release di 

albums = albums[albums.duplicated(subset='id') == False]

In [ ]:
albums.head()

In [353]:
## Now, we are going to get all tracks data
## Spotify provides general informations about Tracks, Audio Features and Audio Analysis

In [354]:
## Creating dataframe to append data

tracks = pd.DataFrame()

In [ ]:
# Creating list of Track ids

tracksList = list(set(albums['track_id'].explode()))

# Redefining batchSize variable

batchSize = 50 # Spotify limit for Track Request is 50

for k in range(0, len(tracksList), batchSize):
    tracksData = sp.tracks(tracksList[k:k + batchSize])
    tracksData = pd.DataFrame.from_dict(tracksData['tracks'])
    
    tracks = pd.concat([tracks, tracksData])
    
    print("Successfully got ", round((k + batchSize) / len(tracksList) * 100, 2), "% of tracks")

In [ ]:
tracks['artist_id'] = tracks['artists'].apply(lambda artists: [artist['id'] for artist in artists])
tracks['artist_href'] = tracks['artists'].apply(lambda artists: [artist['href'] for artist in artists])
tracks['artist_uri'] = tracks['artists'].apply(lambda artists: [artist['uri'] for artist in artists])

tracks['album_id'] = pd.json_normalize(tracks['album'])['id']
tracks['album_href'] = pd.json_normalize(tracks['album'])['href']
tracks['album_uri'] = pd.json_normalize(tracks['album'])['uri']

In [11]:
## Importing Credentials from Google Cloud

from google.cloud import storage
from google.oauth2 import service_account

CREDENTIALS = service_account.Credentials.from_service_account_file(penv.bq_path)
STORAGE = storage.Client(credentials=CREDENTIALS)

In [12]:
# Acessing Bucket Path

bucket = STORAGE.get_bucket(penv.bucket_path)

In [108]:
# Getting currentTimestamp

currentTimestamp = datetime.today().strftime('%Y-%m-%d %X')

# Adding currentTimestamp on file name, so it doesn't overwrite itself. 
# Also, it helps keep track on incremental models

file_name = f"spotify_api_test_data__{currentTimestamp}"

In [123]:
df = artists

In [19]:
## Defining a function called avro_df_prep to prepare the dataframe for the Avro format

def avro_df_prep():

    # pip install fastavro

    from fastavro import writer, parse_schema

    # Converting all columns to string, because Avro doesn't support object type

    columns_to_convert = [  # Lista de colunas definidas no esquema Avro
        'album_type', 'external_urls', 'href',
       'id', 'images', 'name', 'release_date', 'release_date_precision',
       'type', 'uri', 'artists', 'restrictions']

    df[columns_to_convert] = df[columns_to_convert].astype(str)

    # Declaring dataframe schema

    schema = {
        'name': 'spotify'
        , 'type': 'record'
        , 'fields': [
                        {'name': 'album_type', 'type': 'string'}, 
                        {'name': 'total_tracks', 'type': 'int'}, 
                        {'name': 'is_playable', 'type': 'boolean'}, 
                        {'name': 'external_urls', 'type': 'string'},
                        {'name': 'id', 'type': 'string'},
                        {'name': 'images', 'type': 'string'},  
                        {'name': 'name', 'type': 'string'}, 
                        {'name': 'release_date', 'type': 'string'}, 
                        {'name': 'release_date_precision', 'type': 'string'}, 
                        {'name': 'href', 'type': 'string'}, 
                        {'name': 'type', 'type': 'string'}, 
                        {'name': 'uri', 'type': 'string'}, 
                        {'name': 'artists', 'type': 'string'}, 
                        {'name': 'restrictions', 'type': 'string'}, 
                    ]
    }

    parsed_schema = parse_schema(schema)
    records = df.to_dict('records')

    # Writing an Avro file on 'archive' directory

    with open(f'/home/tabas/personal-dev/pyprojects/pipelines/archive/{file_name}.avro', 'wb') as out:
       writer(out, parsed_schema, records)
            

In [ ]:
avro_df_prep()

In [124]:
## Writing Dataframe to Bucket folder with desired file format 

file_formats = [
                #'csv'
                 'parquet'
                #, 'json'
                #, 'orc'
                #, 'avro'
]

for i in range(len(file_formats)):
    
    blob = bucket.blob(f"{penv.bucket_folder}/{file_name}.{file_formats[i]}")
    
    if file_formats[i] == 'csv':
        print("Begin at: ", datetime.today().strftime('%Y-%m-%d %X'))
        blob.upload_from_string(df.to_csv(), '/text/csv')
        print("Sucessfully written in ", file_formats[i])
        print("End at: ", datetime.today().strftime('%Y-%m-%d %X'))
        
    if file_formats[i] == 'parquet':
        print("Begin at: ", datetime.today().strftime('%Y-%m-%d %X'))
        blob.upload_from_string(df.to_parquet(), '/text/plain')
        print("Sucessfully written in ", file_formats[i])
        print("End at: ", datetime.today().strftime('%Y-%m-%d %X'))
        
    if file_formats[i] == 'json':
        print("Begin at: ", datetime.today().strftime('%Y-%m-%d %X'))
        blob.upload_from_string(df.to_json(orient='table'), '/text/plain')
        print("Sucessfully written in ", file_formats[i])
        print("End at: ", datetime.today().strftime('%Y-%m-%d %X'))
        
    if file_formats[i] == 'orc':
        print("Begin at: ", datetime.today().strftime('%Y-%m-%d %X'))
        blob.upload_from_string(df.reset_index().to_orc(index=None), '/text/plain')
        print("Sucessfully written in ", file_formats[i])        
        print("End at: ", datetime.today().strftime('%Y-%m-%d %X'))
        
    if file_formats[i] == 'avro':
        print("Begin at: ", datetime.today().strftime('%Y-%m-%d %X'))
        avro_df_prep()
        blob.upload_from_filename(f'/home/tabas/personal-dev/pyprojects/pipelines/archive/{file_name}.avro', 'wb', '/text/plain')
        print("Sucessfully written in ", file_formats[i])        
        print("End at: ", datetime.today().strftime('%Y-%m-%d %X'))

Begin at:  2025-02-06 10:59:50
Sucessfully written in  parquet
End at:  2025-02-06 10:59:51


In [17]:
## This section is focused on understanding the data to find if further treatment is necessary
## Throuhgout the notebook, I've made some treatments base on what I've seen executiong the commands here in the notebook
## , but now I'm going to bring each dataframe (releases, artists, albums, tracks) to BigQuery and do some querying there

In [18]:
""" Releases (before removing duplicated values)

Run made 06/02/2025 10:05

- album_type: 'single', 'album', 'compilation';
- release_date: minimum value is 2 weeks before today;
- release_date_precision: take only value 'day';
- type: take only value 'album';


- Querying the distinct data, we get 1428 unique releas1es and 1507 unique artists;
SELECT 
  COUNT(DISTINCT id) AS qtd_release
  , COUNT(DISTINCT element) AS qtd_artists
FROM df_releases` 
  , UNNEST(artist_id.list) AS art_id
LIMIT 1000

- Checking if duplicated ids have indeed all the same values on all the columns id = '0hYiEucmpqclKoXxkBiIPS' as example) 
SELECT 
  *
FROM `tabas-dw-stg.stg_juliana.test_releases` 
  , UNNEST(artist_id.list) AS art_id
WHERE id = '0hYiEucmpqclKoXxkBiIPS' 

"""

" Releases (before removing duplicated values)\n\n- album_type: 'single', 'album', 'compilation';\n- release_date: minimum value is 2 weeks before today;\n- release_date_precision: take only value 'day';\n- type: take only value 'album';\n\n- Querying the distinct data, we get 1428 unique releases and 1507 unique artists;\nSELECT \n  COUNT(DISTINCT id) AS qtd_release\n  , COUNT(DISTINCT element) AS qtd_artists\nFROM df_releases` \n  , UNNEST(artist_id.list) AS art_id\nLIMIT 1000\n\n"

In [ ]:
""" Artists 

Run made 06/02/2025 10:07

- genres: list of values - initially, I treated this field extracting the values from the list and maintaining 
        them solely on comma separated values, but on BigQuery it's possible to unnest values easily, so I 
        changed it back to original format;
- type: take only value 'artist';
- followers: total followers the artist has on Spotify profile - initially, the amount was divergent from what it was
        being shown on Spotify's page. I used json_normalize() to treat this field, but on investigation I've discovered 
        that it was atributing the wrong values. Using lambda function, it worked;
- popularity: goes from 0 to 100 - the artist's popularity is calculated from the popularity of all the artist's tracks.

- After running artists dataframe, we realize that it returned 1507 artists (as expected from above).

"""

In [125]:
artists[artists['id'] == '6xlRSRMLgZbsSNd0BMobwy']

,id,href,uri,spotify_url,type,name,followers,popularity,genres,images,extractionTimestamp
32,6xlRSRMLgZbsSNd0BMobwy,https://api.spotify.com/v1/artists/6xlRSRMLgZb...,spotify:artist:6xlRSRMLgZbsSNd0BMobwy,https://open.spotify.com/artist/6xlRSRMLgZbsSN...,artist,DENNIS,2267224,67,"[funk, brazilian funk, funk carioca, funk pop,...",[{'url': 'https://i.scdn.co/image/ab6761610000...,2025-02-06 10:58:53


In [44]:
sp.artist('6xlRSRMLgZbsSNd0BMobwy')

{'external_urls': {'spotify': 'https://open.spotify.com/artist/6xlRSRMLgZbsSNd0BMobwy'},
 'followers': {'href': None, 'total': 2267224},
 'genres': ['funk',
  'brazilian funk',
  'funk carioca',
  'funk pop',
  'funk melody'],
 'href': 'https://api.spotify.com/v1/artists/6xlRSRMLgZbsSNd0BMobwy',
 'id': '6xlRSRMLgZbsSNd0BMobwy',
 'images': [{'url': 'https://i.scdn.co/image/ab6761610000e5eb4ca8b241787391c0d625a162',
   'height': 640,
   'width': 640},
  {'url': 'https://i.scdn.co/image/ab676161000051744ca8b241787391c0d625a162',
   'height': 320,
   'width': 320},
  {'url': 'https://i.scdn.co/image/ab6761610000f1784ca8b241787391c0d625a162',
   'height': 160,
   'width': 160}],
 'name': 'DENNIS',
 'popularity': 67,
 'type': 'artist',
 'uri': 'spotify:artist:6xlRSRMLgZbsSNd0BMobwy'}